<a href="https://colab.research.google.com/github/hyesungKomet/rokaf_ai/blob/main/Elice_1_2_Tensorflow_%26_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 딥러닝 모델의 학습 방법

**예측값과 실제값 간의 오차값을 최소화하는 모델의 인자 찾기!**

**Loss function을 최소화하는 가중치를 찾기 위한 최적화 알고리즘 적용!**

## 학습 순서

1. 순전파 (Forward propagation)

학습용 feature 데이터를 입력하여 예측값 구하기

입력 값을 바탕으로 출력 값 계산

2. Loss 계산

예측값과 실제값 사이의 오차 구하기

-> Loss function 구함(MSE, Cross Entropy등등)

-> 최적화는 어떻게? 경사 하강법!!!

3. 역전파 (Back propagation)

Loss를 줄일 수 있는 가중치 업데이트하기
각 가중치의 gradient를 구하기 위해서는 직전의 계산값이 필요함

4. 1~3 반복하여 Loss 최소로 하는 가중치 구하기

1~3으로 가중치가 업데이트됨

![gradient descent](https://i.ytimg.com/vi/b4Vyma9wPHo/maxresdefault.jpg)

![back propagation](https://i.ytimg.com/vi/iyn2zdALii8/maxresdefault.jpg)

SyntaxError: ignored

# TensorFlow

딥러닝 모델 구현 순서
1. 데이터 전처리하기
2. 딥러닝 모델 구축
3. 모델 학습시키기
4. 평가 및 예측하기

1. 데이터 전처리하기
* Tensor 형태의 데이터 입력받는다(Numpy나 Pandas로 받은 데이터도 tf에서 Tensor 형태로 변환됨)
* Epoch: 전체 데이터 셋으로 한번 학습 완료
* Batch: 나눠진 데이터 묶음( == mini-batch)
* Interation: epoch를 나누어서 실행
* ex) data: 1000, Batch size = 10  
1 epoch = data / Batch size = 100 iteration

In [38]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

np.random.seed(100)
tf.random.set_seed(100)

# 데이터를 DataFrame 형태로 불러 옵니다.
# 굳이 얘를 넣고싶진 않아서 걍 코드만....
df = pd.read_csv("drive/MyDrive/Advertising.csv")

# DataFrame 데이터 샘플 5개를 출력합니다.
print('원본 데이터 샘플 :')
print(df.head(),'\n')

# 의미없는 변수는 삭제합니다.
df = df.drop(columns=['Unnamed: 0'])

"""
1. Sales 변수는 label 데이터로 Y에 저장하고 나머진 X에 저장합니다.
"""
X = df.drop(columns=['Sales'])
Y = df['Sales']

train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=0.3)

"""
2. 학습용 데이터를 tf.data.Dataset 형태로 변환합니다.
   from_tensor_slices 함수를 사용하여 변환하고 batch를 수행하게 합니다.
"""
train_ds = tf.data.Dataset.from_tensor_slices((train_X.values, train_Y.values))
train_ds = train_ds.shuffle(len(train_X)).batch(batch_size=5)

# 하나의 batch를 뽑아서 feature와 label로 분리합니다.
# x, y로 나누어져 있는 거 생각하면 된다
[(train_features_batch, label_batch)] = train_ds.take(1)

# batch 데이터를 출력합니다.
print('\nFB, TV, Newspaper batch 데이터:\n',train_features_batch)
print('Sales batch 데이터:',label_batch)


원본 데이터 샘플 :
   Unnamed: 0     FB    TV  Newspaper  Sales
0           1  230.1  37.8       69.2   22.1
1           2   44.5  39.3       45.1   10.4
2           3   17.2  45.9       69.3    9.3
3           4  151.5  41.3       58.5   18.5
4           5  180.8  10.8       58.4   12.9 


FB, TV, Newspaper batch 데이터:
 tf.Tensor(
[[296.4  36.3 100.9]
 [228.   37.7  32. ]
 [  5.4  29.9   9.4]
 [ 57.5  32.8  23.5]
 [240.1   7.3   8.7]], shape=(5, 3), dtype=float64)
Sales batch 데이터: tf.Tensor([23.8 21.5  5.3 11.8 13.2], shape=(5,), dtype=float64)


2. 딥러딩 모델 구축하기: 고수준 API 활용(feat. Keras)
* 방법 1
  * 모델 클래스 객체 생성
  tf.kears.models.Sequential()
  * 모델의 각 Layer 구성
  tf.keras.layers.Dense(units, activation)
  units: 레이어 안의 Node 수  
  activation: 적용할 활성화 함수 결정  
  Input Layer은 input_shape, input_dim 인수 결정
* 방법 2
  * 모델에 Layer추가하기
  [model].add(tf.kears.layers.Dense(units, activation))


In [ ]:

"""
1. tf.keras.models.Sequential()를 활용하여 신경망 모델을 생성합니다.
   자유롭게 layers를 쌓고 마지막 layers는 노드 수를 1개로 설정합니다.
   회귀 분석이기에 마지막이 한 개인 것!
"""
model = tf.keras.models.Sequential([
    # FB, TV, Newspaper로 Sales 예측이니 input shape은 (3, )인 것!
    tf.keras.layers.Dense(10, input_shape=(3,)),
    tf.keras.layers.Dense(1)
    ])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                40        
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 51
Trainable params: 51
Non-trainable params: 0
_________________________________________________________________
None


3. 딥러닝 모델 학습시키기
* 모델 학습 방식 설정하는 함수
[model].compile(optimizer, loss)  
optimizer: 모델 학습 최적화 방법  
ex) gradient descent, SGD, momentum, adam, etc  
loss: 손실 함수 설정  
ex) 회귀: MSE(Mean Square Error) 분류: Cross Entropy 등등

* 모델을 학습시키는 함수  
[model].fit(x, y, epochs)  
x: 학습 데이터
y: 학습 데이터의 label

In [ ]:
"""
1. 학습용 데이터를 바탕으로 모델의 학습을 수행합니다.
    
step1. compile 메서드를 사용하여 최적화 모델 설정합니다.
       loss는 mean_squared_error, optimizer는 adam으로 설정합니다.
       
step2. fit 메서드를 사용하여 Dataset으로 변환된 학습용 데이터를 학습합니다.
       epochs는 100으로 설정합니다.
"""
# loss는 회귀에서는 일반적으로 MSE인 ‘mean_squared_error’, 
# 분류에서는 ‘sparse_categorical_crossentropy’ 를 주로 사용
model.compile(loss='mean_squared_error', optimizer='adam')
history = model.fit(train_ds, epochs=100, verbose=1)
# verbose 0은 표기없음 1은 진행바, 2는 에포크당 한 줄 출력

Epoch 1/100
28/28 [==============================] - 1s 1ms/step - loss: 1746.2518
Epoch 2/100
28/28 [==============================] - 0s 1ms/step - loss: 323.6100
Epoch 3/100
28/28 [==============================] - 0s 1ms/step - loss: 158.4601
Epoch 4/100
28/28 [==============================] - 0s 1ms/step - loss: 109.7061
Epoch 5/100
28/28 [==============================] - 0s 1ms/step - loss: 75.6144
Epoch 6/100
28/28 [==============================] - 0s 1ms/step - loss: 52.3664
Epoch 7/100
28/28 [==============================] - 0s 1ms/step - loss: 36.6032
Epoch 8/100
28/28 [==============================] - 0s 1ms/step - loss: 26.3099
Epoch 9/100
28/28 [==============================] - 0s 1ms/step - loss: 20.1578
Epoch 10/100
28/28 [==============================] - 0s 1ms/step - loss: 15.9902
Epoch 11/100
28/28 [==============================] - 0s 2ms/step - loss: 13.1412
Epoch 12/100
28/28 [==============================] - 0s 1ms/step - loss: 11.0550
Epoch 13/100
28/28 [

4. 평가 및 예측하기
* 모델 평가하는 메소드  
[model].evaluate(x, y)  
x: 테스트 데이터  
y: 테스트 데이터의 label
* 모델 예측하는 메소드  
[model].predict(x)  
x: 예측하고자 하는 데이터

In [ ]:
"""
1. evaluate 메서드를 사용하여 테스트용 데이터의 loss 값을 계산합니다.
"""
loss = model.evaluate(test_X, test_Y, verbose=1)
# Tensor로 안해도 evaluate가 자동으로 변환해줌
# batch 설정 안한다 그건 학습할 때만!

"""
2. predict 메서드를 사용하여 테스트용 데이터의 예측값을 계산합니다.
"""
predictions = model.predict(test_X)

# 결과를 출력합니다.
print("테스트 데이터의 Loss 값: ", loss)
for i in range(5):
    print("%d 번째 테스트 데이터의 실제값: %f" % (i, test_Y.iloc[i]))
    print("%d 번째 테스트 데이터의 예측값: %f" % (i, predictions[i][0]))


2/2 [==============================] - 0s 12ms/step - loss: 2.5349
테스트 데이터의 Loss 값:  2.5348823070526123
0 번째 테스트 데이터의 실제값: 6.600000
0 번째 테스트 데이터의 예측값: 10.425418
1 번째 테스트 데이터의 실제값: 20.700000
1 번째 테스트 데이터의 예측값: 19.977503
2 번째 테스트 데이터의 실제값: 17.200000
2 번째 테스트 데이터의 예측값: 17.103909
3 번째 테스트 데이터의 실제값: 19.400000
3 번째 테스트 데이터의 예측값: 19.590160
4 번째 테스트 데이터의 실제값: 21.800000
4 번째 테스트 데이터의 예측값: 21.168644


## Iris 신경망 모델로 분류하기

In [40]:
from sklearn.datasets import load_iris

X, Y = load_iris(return_X_y = True)

df = pd.DataFrame(X, columns=['꽃받침 길이', '꽃받침 넓이', '꽃잎 길이', '꽃잎 넓이'])
df['클래스'] = Y

df.head(10)

,꽃받침 길이,꽃받침 넓이,꽃잎 길이,꽃잎 넓이,클래스
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
5,5.4,3.9,1.7,0.4,0
6,4.6,3.4,1.4,0.3,0
7,5.0,3.4,1.5,0.2,0
8,4.4,2.9,1.4,0.2,0
9,4.9,3.1,1.5,0.1,0


In [41]:
X = df.drop(columns=['클래스'])
Y = df['클래스']

In [42]:
# train, test data separation
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=0.2, random_state=42)

In [43]:
print(train_X.values.shape)
print(train_Y.shape)

(120, 4)
(120,)


In [44]:
train_ds = tf.data.Dataset.from_tensor_slices((train_X.values, train_Y))
train_ds = train_ds.shuffle(len(train_X)).batch(batch_size=5)

In [45]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(10, input_dim=4),
  tf.keras.layers.Dense(3, activation='softmax')
])

In [46]:
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])
history = model.fit(train_ds, epochs=100, verbose=1)

Epoch 1/100
24/24 [==============================] - 0s 1ms/step - loss: 3.5424 - accuracy: 0.3333
Epoch 2/100
24/24 [==============================] - 0s 1ms/step - loss: 2.3964 - accuracy: 0.4667
Epoch 3/100
24/24 [==============================] - 0s 2ms/step - loss: 1.5727 - accuracy: 0.6583
Epoch 4/100
24/24 [==============================] - 0s 1ms/step - loss: 0.9641 - accuracy: 0.6250
Epoch 5/100
24/24 [==============================] - 0s 2ms/step - loss: 0.8294 - accuracy: 0.4000
Epoch 6/100
24/24 [==============================] - 0s 2ms/step - loss: 0.7850 - accuracy: 0.4500
Epoch 7/100
24/24 [==============================] - 0s 1ms/step - loss: 0.7495 - accuracy: 0.4333
Epoch 8/100
24/24 [==============================] - 0s 2ms/step - loss: 0.7210 - accuracy: 0.5417
Epoch 9/100
24/24 [==============================] - 0s 2ms/step - loss: 0.6917 - accuracy: 0.5417
Epoch 10/100
24/24 [==============================] - 0s 1ms/step - loss: 0.6617 - accuracy: 0.6167
Epoch 11/

In [47]:
loss, acc = model.evaluate(test_X, test_Y)

1/1 [==============================] - 0s 329ms/step - loss: 0.1415 - accuracy: 1.0000


In [48]:
prediction = model.predict(test_X)

In [49]:
print("test data accuracy: ", acc)
for i in range(10):
  print("{}번째 test data answer: \t{}".format(i, test_Y.iloc[i]))
  print("{}번째 test data prediction: \t{}".format(i, np.argmax(prediction[i])))

test data accuracy:  1.0
0번째 test data answer: 	1
0번째 test data prediction: 	1
1번째 test data answer: 	0
1번째 test data prediction: 	0
2번째 test data answer: 	2
2번째 test data prediction: 	2
3번째 test data answer: 	1
3번째 test data prediction: 	1
4번째 test data answer: 	1
4번째 test data prediction: 	1
5번째 test data answer: 	0
5번째 test data prediction: 	0
6번째 test data answer: 	1
6번째 test data prediction: 	1
7번째 test data answer: 	2
7번째 test data prediction: 	2
8번째 test data answer: 	1
8번째 test data prediction: 	1
9번째 test data answer: 	1
9번째 test data prediction: 	1
